In [1]:
import urllib.parse

CLIENT_ID = "J58PIDNM3UQCNE85UVRMSSIO7F8HUNGD7KPNR9004IK3LIKE6GNBAPDBIDHMOUH1"
REDIRECT_URI = "https://ai-resume-oauth-server.onrender.com/callback"
STATE = "12345"  # любая строка для защиты от CSRF

params = {
    "response_type": "code",
    "client_id": CLIENT_ID,
    "redirect_uri": REDIRECT_URI,
    "state": STATE
}
auth_url = "https://hh.ru/oauth/authorize?" + urllib.parse.urlencode(params)
print("Перейдите по ссылке, авторизуйтесь, и вас перенаправит сюда с ?code=…:")
print(auth_url)


Перейдите по ссылке, авторизуйтесь, и вас перенаправит сюда с ?code=…:
https://hh.ru/oauth/authorize?response_type=code&client_id=J58PIDNM3UQCNE85UVRMSSIO7F8HUNGD7KPNR9004IK3LIKE6GNBAPDBIDHMOUH1&redirect_uri=https%3A%2F%2Fai-resume-oauth-server.onrender.com%2Fcallback&state=12345


In [2]:
import requests

def get_tokens(auth_code):
    data = {
        "grant_type": "authorization_code",
        "client_id": CLIENT_ID,
        "client_secret": "IGBU6EJV14587HUCAAJRRC0B63SNSUDL7J2PR39RR438SPECU64LAQ15EVNEJLL2",
        "code": auth_code,
        "redirect_uri": REDIRECT_URI
    }
    resp = requests.post("https://hh.ru/oauth/token", data=data)
    resp.raise_for_status()
    tokens = resp.json()
    # tokens: {"access_token": "...", "refresh_token": "...", "expires_in": ...}
    return tokens

# Вставьте сюда полученный code
auth_code = "U17FPIR9E3H3BU7OCMUNU2N523000HKO7GQ6R6O7LM55103TOVMKLFM1JRBO2UAN"
tokens = get_tokens(auth_code)
print(tokens)


{'access_token': 'USERLVMNFD8ENI5SLFSERIQ445AVCIP4V4908EDSN534B03QOJPKTNQK2Q5H62TJ', 'token_type': 'bearer', 'refresh_token': 'USERR67J0KNH7JGHEB7STQIEVCGGVG1QGF54LVQ6D2PLU0A62KJ948PD5B22SN9E', 'expires_in': 1209599}


In [3]:
def get_my_resumes(access_token):
    url = "https://api.hh.ru/resumes/mine"
    headers = {"Authorization": f"Bearer {access_token}"}
    resp = requests.get(url, headers=headers)
    resp.raise_for_status()
    return resp.json()

resumes_data = get_my_resumes(tokens["access_token"])
print(resumes_data)


{'items': [{'last_name': 'Немов', 'first_name': 'Максим', 'middle_name': 'Юрьевич', 'title': 'NLP LLM Engineer', 'created_at': '2025-06-04T08:19:15+0300', 'updated_at': '2025-07-10T13:43:24+0300', 'area': {'id': '1', 'name': 'Москва', 'url': 'https://api.hh.ru/areas/1'}, 'age': 24, 'gender': {'id': 'male', 'name': 'Мужской'}, 'salary': {'amount': 250000, 'currency': 'RUR'}, 'photo': {'id': '184744592', 'small': 'https://img.hhcdn.ru/photo/788192684.jpeg?t=1752231826&h=fKYXVUanQenT7fbIaFPlEg', 'medium': 'https://img.hhcdn.ru/photo/788192685.jpeg?t=1752231826&h=8z2LrBLt9kUMIELE3Kqhzg', '40': 'https://img.hhcdn.ru/photo/788192683.jpeg?t=1752231826&h=1ylBHkJPlZErEsg-AA98SQ', '100': 'https://img.hhcdn.ru/photo/788192684.jpeg?t=1752231826&h=fKYXVUanQenT7fbIaFPlEg', '500': 'https://img.hhcdn.ru/photo/788192685.jpeg?t=1752231826&h=8z2LrBLt9kUMIELE3Kqhzg'}, 'total_experience': {'months': 41}, 'certificate': [{'owner': None, 'type': 'custom', 'title': 'Специализиция "Data Engineering"', 'achieve

In [4]:
import json

def get_resume_by_id(access_token, resume_id):
    url = f"https://api.hh.ru/resumes/{resume_id}"
    headers = {"Authorization": f"Bearer {access_token}"}
    resp = requests.get(url, headers=headers)
    resp.raise_for_status()
    return resp.json()

# Пример использования
resume_id = resumes_data["items"][0]["id"]
resume_obj = get_resume_by_id(tokens["access_token"], resume_id)

# Сохраним в файл
with open("resume_nlp.json", "w", encoding="utf-8") as f:
    json.dump(resume_obj, f, ensure_ascii=False, indent=2)
